In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [8]:
print(X)




[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [9]:
#label encoding the gender column
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2]= le.fit_transform(X[:,2])

In [10]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [12]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [13]:
from sklearn.model_selection import train_test_split


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [16]:
X_test.max()

250898.09

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
X_train.max()

4.703418381363563

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 

In [22]:
ann= Sequential()

ann.add(Dense(6,activation='relu'))
ann.add(Dense(6,activation='relu'))
ann.add(Dense(1,activation='sigmoid'))


In [23]:
ann.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100,verbose=2)

Epoch 1/100
250/250 - 1s - loss: 0.5313 - accuracy: 0.7806
Epoch 2/100
250/250 - 0s - loss: 0.4573 - accuracy: 0.7960
Epoch 3/100
250/250 - 0s - loss: 0.4373 - accuracy: 0.7977
Epoch 4/100
250/250 - 0s - loss: 0.4259 - accuracy: 0.8008
Epoch 5/100
250/250 - 0s - loss: 0.4180 - accuracy: 0.8011
Epoch 6/100
250/250 - 0s - loss: 0.4115 - accuracy: 0.8065
Epoch 7/100
250/250 - 0s - loss: 0.4064 - accuracy: 0.8136
Epoch 8/100
250/250 - 0s - loss: 0.4009 - accuracy: 0.8159
Epoch 9/100
250/250 - 0s - loss: 0.3969 - accuracy: 0.8189
Epoch 10/100
250/250 - 0s - loss: 0.3938 - accuracy: 0.8273
Epoch 11/100
250/250 - 0s - loss: 0.3908 - accuracy: 0.8324
Epoch 12/100
250/250 - 0s - loss: 0.3883 - accuracy: 0.8339
Epoch 13/100
250/250 - 0s - loss: 0.3864 - accuracy: 0.8367
Epoch 14/100
250/250 - 0s - loss: 0.3842 - accuracy: 0.8361
Epoch 15/100
250/250 - 0s - loss: 0.3821 - accuracy: 0.8380
Epoch 16/100
250/250 - 0s - loss: 0.3802 - accuracy: 0.8393
Epoch 17/100
250/250 - 0s - loss: 0.3780 - accura

In [25]:
# Use our ANN model to predict if the customer with the following informations will leave the bank:

# Geography: France

# Credit Score: 600

# Gender: Male

# Age: 40 years old

# Tenure: 3 years

# Balance: $ 60000

# Number of Products: 2

# Does this customer have a credit card? Yes

# Is this customer an Active Member: Yes

# Estimated Salary: $ 50000

# So, should we say goodbye to that customer?

In [28]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

[[False]]


In [29]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1521   74]
 [ 192  213]]


0.867

In [31]:
ann.save('Churn_Modelling_model.h5')